# Radius Search Practice
To be used for bare critical mass calculation
<br >Created on 18 October 2022

In [1]:
# prepare the environment
import openmc
import openmc.model
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## Create Parametrized Model

In [2]:
def build_model(radius):
    
    # define materials
    # HEU U.S. Average taken from PNNL Material compendium
    fuel = openmc.Material(name="Bare Sphere")
    fuel.set_density('g/cc', 18.724760)
    fuel.add_nuclide('U234', 0.009849, 'ao')
    fuel.add_nuclide('U235', 0.931550, 'ao')
    fuel.add_nuclide('U236', 0.004500, 'ao')
    fuel.add_nuclide('U238', 0.054150, 'ao')
    
    materials = openmc.Materials([fuel])
    
    # create sphere with radius parameter
    sphere_radius = openmc.Sphere(x0=0,y0=0,z0=0,r=radius, boundary_type='vacuum', name='BCM')
    
    # create core cell
    core_cell = openmc.Cell(name='Bare Critical Sphere')
    core_cell.fill = fuel
    core_cell.region = -sphere_radius
    
    # create universe geometry
    root_universe = openmc.Universe(name='root universe')
    root_universe.add_cells([core_cell])
    
    geometry = openmc.Geometry(root_universe)
    # define criticality settings
    settings = openmc.Settings()
    settings.run_mode = 'eigenvalue' # keff calculation
    settings.particles = 5000 # particles per batch (mo betta)
    settings.batches = 250 # number of batches
    settings.inactive = 50 # inactive batches
    
    settings.output = {'tallies': False}
    
    model = openmc.model.Model(geometry,materials,settings)
    
    return model

## Conduct Radius Search

In [3]:
crit_r, guesses, keffs = openmc.search_for_keff(build_model, bracket=[1,100],
                                                tol=1e-3, print_iterations=True,
                                               run_args={'output':False})
print('The bare critical sphere radius is %7.4f cm.' % crit_r)

Iteration: 1; Guess of 1.00e+00 produced a keff of 0.11862 +/- 0.00009
Iteration: 2; Guess of 1.00e+02 produced a keff of 2.22317 +/- 0.00049
Iteration: 3; Guess of 5.05e+01 produced a keff of 2.12467 +/- 0.00075
Iteration: 4; Guess of 2.58e+01 produced a keff of 1.86094 +/- 0.00083
Iteration: 5; Guess of 1.34e+01 produced a keff of 1.36645 +/- 0.00077
Iteration: 6; Guess of 7.19e+00 produced a keff of 0.84045 +/- 0.00059
Iteration: 7; Guess of 1.03e+01 produced a keff of 1.13415 +/- 0.00075
Iteration: 8; Guess of 8.73e+00 produced a keff of 0.99613 +/- 0.00070
Iteration: 9; Guess of 9.51e+00 produced a keff of 1.06613 +/- 0.00073
Iteration: 10; Guess of 9.12e+00 produced a keff of 1.03214 +/- 0.00069
Iteration: 11; Guess of 8.93e+00 produced a keff of 1.01502 +/- 0.00066
Iteration: 12; Guess of 8.83e+00 produced a keff of 1.00435 +/- 0.00068
Iteration: 13; Guess of 8.78e+00 produced a keff of 0.99961 +/- 0.00067
Iteration: 14; Guess of 8.81e+00 produced a keff of 1.00332 +/- 0.00064
I

In [6]:
def build_model2(radius):
    
    # define materials
    # HEU U.S. Average taken from PNNL Material compendium
    fuel = openmc.Material(name="Bare Sphere")
    fuel.set_density('g/cc', 18.724760)
    fuel.add_element('U',1, enrichment=7)
    
    materials = openmc.Materials([fuel])
    
    # create sphere with radius parameter
    sphere_radius = openmc.Sphere(x0=0,y0=0,z0=0,r=radius, boundary_type='vacuum', name='BCM')
    
    # create core cell
    core_cell = openmc.Cell(name='Bare Critical Sphere')
    core_cell.fill = fuel
    core_cell.region = -sphere_radius
    
    # create universe geometry
    root_universe = openmc.Universe(name='root universe')
    root_universe.add_cells([core_cell])
    
    geometry = openmc.Geometry(root_universe)
    # define criticality settings
    settings = openmc.Settings()
    settings.run_mode = 'eigenvalue' # keff calculation
    settings.particles = 5000 # particles per batch (mo betta)
    settings.batches = 250 # number of batches
    settings.inactive = 50 # inactive batches
    
    settings.output = {'tallies': False}
    
    model = openmc.model.Model(geometry,materials,settings)
    
    return model

In [8]:
crit_r1, guesses1, keffs1 = openmc.search_for_keff(build_model2, bracket=[1,100],
                                                tol=5e-4, print_iterations=True,
                                               run_args={'output':False})
print('The bare critical sphere radius is %7.4f cm.' % crit_r1)

/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 7 was given for Material ID="35". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)
/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 7 was given for Material ID="36". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


Iteration: 1; Guess of 1.00e+00 produced a keff of 0.03679 +/- 0.00004


/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 7 was given for Material ID="37". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


Iteration: 2; Guess of 1.00e+02 produced a keff of 1.06237 +/- 0.00069


/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 7 was given for Material ID="38". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


...

Iteration: 12; Guess of 6.39e+01 produced a keff of 0.99790 +/- 0.00068


/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 7 was given for Material ID="48". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


Iteration: 13; Guess of 6.40e+01 produced a keff of 0.99857 +/- 0.00067


/home/m231326/anaconda3/envs/omc/lib/python3.10/site-packages/openmc/material.py:640: UserWarning: A uranium enrichment of 7 was given for Material ID="49". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


Iteration: 14; Guess of 6.40e+01 produced a keff of 1.00001 +/- 0.00065
The bare critical sphere radius is 64.0110 cm.
